# Create csv file from pre processed text files:

In [3]:
import os
import pandas as pd

data_dir = '/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/price/preprocessed'
os.makedirs(os.path.join(data_dir, 'csv_test/'))

for stock_name in os.listdir(data_dir):
    txt_stockprice_file = os.path.join(data_dir, stock_name)
    if txt_stockprice_file.endswith('txt'):
        csv_stockprice_file = os.path.join(data_dir, 'csv_test/', stock_name.replace('.txt', '.csv'))
        #csv_stockprice_file = f"csv/{txt_stockprice_file.replace('.txt', '.csv')}"
        df = pd.read_csv(
                    txt_stockprice_file, 
                    delimiter="\t", 
                    header=None, 
                    names=["Date", "Movement_Percent", "Open", "High", "Low", "Close", "Volume"]
                )
        
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.sort_values(by="Date")

        df.to_csv(csv_stockprice_file, index=False)



### Convert tweets files to proper json

In [7]:
import os
import json

# Input folder containing the improperly formatted JSON files
input_folder = "/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/tweet/preprocessed"
output_root = "/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/tweet/preprocessed-json-test/"



for stock in os.listdir(input_folder):
    stock_folder = os.path.join(input_folder, stock)
    for date in os.listdir(stock_folder):
        output_folder = os.path.join(output_root, stock)
        input_file = os.path.join(input_folder, stock, date)
        if date.startswith("20"):
            os.makedirs(output_folder, exist_ok=True)
            output_file = os.path.join(output_folder, f'{date}.json')
            # Open the input file and create the output JSON file
            with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
                json_list = []
                for line in infile:
                    try:
                        # Parse the line as JSON
                        parsed_line = json.loads(line.strip())
                        json_list.append(parsed_line)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding line in {input_file}: {e}")
                
                # Write the proper JSON to the output file
                json.dump(json_list, outfile, indent=4)

### Generate dataset with sentiment for each tweet.

In [ ]:
import os
import json
import torch
from transformers import BertTokenizer, AutoTokenizer, AutoModelForSequenceClassification
from config import cfg
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(cfg.dataset_loader_args['sentiment_model'])
sentiment_model = AutoModelForSequenceClassification.from_pretrained(cfg.dataset_loader_args['sentiment_model']).to('cuda')
sentiment_model.eval()
def get_sentiment_from_tweets(list_of_tweets):
        #print(list_of_tweets)
    

    encoded_inputs = tokenizer(
                list_of_tweets,
                return_tensors='pt',
                padding=True,
                truncation=True,
                max_length=30  # Adjust max_length as needed
            ).to('cuda')

    with torch.no_grad():
        outputs = sentiment_model(**encoded_inputs)
        predictions = outputs.logits.argmax(dim=1)
    sentiment_scores = predictions.cpu().numpy() + 1
    return sentiment_scores

# Input folder containing the improperly formatted JSON files
input_folder = "/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/tweet/preprocessed-test"
output_root = "/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/tweet/preprocessed-sentiment-json-test2/"



for stock in os.listdir(input_folder):
    stock_folder = os.path.join(input_folder, stock)
    for date in os.listdir(stock_folder):
        output_folder = os.path.join(output_root, stock)
        input_file = os.path.join(input_folder, stock, date)
        if date.startswith("20"):
            os.makedirs(output_folder, exist_ok=True)
            output_file = os.path.join(output_folder, f'{date}.json')
            if os.path.exists(output_file):
                continue
            # Open the input file and create the output JSON file
            with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
                json_list = []
                for line in infile:
                    try:
                        # Parse the line as JSON
                        parsed_line = json.loads(line.strip())
                        json_list.append(parsed_line)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding line in {input_file}: {e}")
                for tweet in json_list:
                    sentiment_score = get_sentiment_from_tweets([" ".join(tweet['text'])])
                    tweet['sentiment'] = int(sentiment_score[0])
                # Write the proper JSON to the output file
                json.dump(json_list, outfile, indent=4)

KeyboardInterrupt: 